In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install torch torchvision transformers opencv-python pycocotools roboflow matplotlib tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 2.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [2]:
import torch
import torchvision
import os
import cv2
from torch.utils.data import Dataset, DataLoader
from transformers import SwinModel
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from transformers import SamModel, SamProcessor
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
from roboflow import Roboflow

rf = Roboflow(api_key="vQsG0k1QDxbi74ZD03Wp")  # Replace with your API key
project = rf.workspace("fire-detection-4mpgq").project("root-cannel")  
dataset = project.version(5).download("yolov5")  # Adjust the version if necessary
print (dataset)

loading Roboflow workspace...
loading Roboflow project...


In [6]:
image_dir = f"./{project.name}-{project.version}/train/images"
annotation_dir = f"./{project.name}-{project.version}/train/labels"
print (image_dir)

./Root cannel-<bound method Project.version of <roboflow.core.project.Project object at 0x7d2924d98550>>/train/images


In [8]:


class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_dir):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        ann_path = os.path.join(self.annotation_dir, img_name.replace('.jpg', '.txt'))

        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = torch.tensor(image).permute(2, 0, 1).float() / 255.0

        # Load annotations (YOLO format)
        boxes, labels = [], []
        with open(ann_path, "r") as file:
            for line in file.readlines():
                data = list(map(float, line.strip().split()))
                label, x_center, y_center, width, height = data
                x1 = (x_center - width / 2) * image.shape[2]
                y1 = (y_center - height / 2) * image.shape[1]
                x2 = (x_center + width / 2) * image.shape[2]
                y2 = (y_center + height / 2) * image.shape[1]
                boxes.append([x1, y1, x2, y2])
                labels.append(int(label))

        target = {"boxes": torch.tensor(boxes, dtype=torch.float32), "labels": torch.tensor(labels, dtype=torch.int64)}
        return image, target


In [9]:

swin_backbone = SwinModel.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
backbone = swin_backbone
backbone.out_channels = 768


config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

In [10]:

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),) * 5)

model = FasterRCNN(backbone, num_classes=10, rpn_anchor_generator=anchor_generator)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_fe

In [11]:

sam_model = SamModel.from_pretrained("facebook/sam-vit-huge")
sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")
sam_model.to(device)


config.json:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

SamModel(
  (shared_image_embedding): SamPositionalEmbedding()
  (vision_encoder): SamVisionEncoder(
    (patch_embed): SamPatchEmbeddings(
      (projection): Conv2d(3, 1280, kernel_size=(16, 16), stride=(16, 16))
    )
    (layers): ModuleList(
      (0-31): 32 x SamVisionLayer(
        (layer_norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): SamVisionAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (layer_norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (mlp): SamMLPBlock(
          (lin1): Linear(in_features=1280, out_features=5120, bias=True)
          (lin2): Linear(in_features=5120, out_features=1280, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (neck): SamVisionNeck(
      (conv1): Conv2d(1280, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (layer_norm1): SamLayerNor

In [12]:
dataset = CustomDataset(image_dir, annotation_dir)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


FileNotFoundError: [Errno 2] No such file or directory: './Root cannel-<bound method Project.version of <roboflow.core.project.Project object at 0x7d2924d98550>>/train/images'

In [ ]:


optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 100

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for images, targets in tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


In [ ]:
torch.save(model.state_dict(), "faster_rcnn_swin.pth")

In [ ]:


def detect_and_segment(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float().to(device)

    model.eval()
    with torch.no_grad():
        detections = model(image_tensor)

    boxes = detections[0]['boxes'].cpu().numpy()

    # Run SAM on the detected image
    inputs = sam_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        masks = sam_model(**inputs).last_hidden_state

    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    plt.imshow(image)
    plt.title("Object Detection + SAM Segmentation")
    plt.axis("off")
    plt.show()

# Test on a sample image
test_image = f"./{project.name}-{project.version}/test/images/example.jpg"
detect_and_segment(test_image)
